In [1]:
import os

In [2]:
%pwd

'u:\\mlflow-dvc-cancer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'u:\\mlflow-dvc-cancer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

# These are the configurations it will return 
# frozen=True will not allow any other functionalities to be added 
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: str 
    local_data_file: Path
    unzip_dir: Path
# This will go in my entity directory of code
    

In [9]:
# Entity
from cnnClassifier.constants import * # This will import the paths for config and param yaml
from cnnClassifier.utils.common import read_yaml, create_directories # Help me to read the yaml files

class ConfigurationManager: # Read the config and param file paths
    def __init__(self, config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # This will create the directory for dataset
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)-> DataIngestionConfig: # Returning 
        config = self.config.data_ingestion # This key has all the values
        create_directories([config.root_dir]) # This will create the root directory
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir, # Storing all the variables
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        ) 
        return data_ingestion_config


In [12]:
import os 
import zipfile 
import gdown 
from cnnClassifier import logger 
from cnnClassifier.utils.common import get_size 

In [17]:
# Componente
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config # This will take the config return class
    
    def download_file(self)->str:
        """ Fetch data from URL """ 
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading the data from {dataset_url} into file {zip_download_dir}")
            # This will find link, make a dir at makedirs and then download the zip file
            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)
            logger.info(f"Downloaded the data from {dataset_url} into file {zip_download_dir}")
        
        except Exception as e:
            raise e
        
    def extract_zip_files(self):
        """ Extract the zip file into a folder """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [18]:
try: 
    config = ConfigurationManager() # Call the ConfigManager
    data_ingestion_config = config.get_data_ingestion_config() # This will have a data_ing method
    data_ingestion = DataIngestion(config=data_ingestion_config) # Will give DataConfig as input
    data_ingestion.download_file() # This will download the data
    data_ingestion.extract_zip_files() # This will unzip the data
except Exception as e:
    raise e

[2024-07-11 01:40:41,732: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-11 01:40:41,736: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-11 01:40:41,738: INFO: common: created directory at: artifacts]
[2024-07-11 01:40:41,741: INFO: common: created directory at: artifacts/data_ingestion]
[2024-07-11 01:40:41,744: INFO: 1693187727: Downloading the data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=9d048c49-563b-4da8-a14b-85dc17022056
To: u:\mlflow-dvc-cancer\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:29<00:00, 1.63MB/s]

[2024-07-11 01:41:14,112: INFO: 1693187727: Downloaded the data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]


In [ ]:
# Now we will be updating my entity folder with config_entity.py
# Then configuration.py in config folder 
# Update the component folder as data_ingestion.py
# Update the pipeline folder as 01_stage_data_ingestion.py
# Now to test code we will populate main.py